In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
cd /content/gdrive/MyDrive/Colab Notebooks/RBM - CF/Recommender-System

/content/gdrive/MyDrive/Colab Notebooks/RBM - CF/Recommender-System


In [1]:
from collaborative_filtering import CF 
from demographic_filtering import DF
from perceptron import Perceptron as pct
from get_data import (
    get_users_data,
    get_rating_base_data,
    get_rating_test_data,
)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch

# Import RBM model script
from rbm import RBM

In [ ]:
# Initialize Weights and Biases
import wandb
wandb.init(entity="phusdt", project="boltzmann_machines_collaborative_filtering")

# Config is a variable that holds and saves hyper-parameters
config = wandb.config  # Initialize config
config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

In [ ]:
# CF 
RATE_TRAIN = get_rating_base_data().values # convert to matrix
RATE_TEST = get_rating_test_data().values # convert to matrix

RATE_TRAIN[:, :2] -= 1 # start from 0
RATE_TEST[:, :2] -= 1

CF = CF(RATE_TRAIN, k=25)
CF.fit()

print('Similar Matrix Works')
print(CF.S)
print('Number of rows: ', CF.S.shape[0])
print('Number of columns: ', CF.S.shape[1])

ids = np.where(RATE_TEST[:, 0] == 0)[0].astype('int32')
real_items_1 = RATE_TEST[(np.where((RATE_TEST[:, 0] == 0) & (RATE_TEST[:, 2] >= 3)))]
predicted_items = []

for row in RATE_TEST[ids, :]:
    predicted_rating = CF.pred(0, row[1])
    if predicted_rating >= 3:
        predicted_items.append(row[1])

print('Items which user 1 actually like: ', real_items_1[:, 1])
print('Items in prediction which user 1 might like ', predicted_items)


n_test = RATE_TEST.shape[0]
correct_items_count = 0
real_items_user_like_count = len(np.where(RATE_TEST[:, 2] >= 3)[0].astype(np.int32))

user_id = 0
while user_id < CF.n_users:
    ids = np.where(RATE_TEST[:, 0] == user_id)[0].astype('int32')
    real_items = RATE_TEST[(np.where((RATE_TEST[:, 0] == user_id) & (RATE_TEST[:, 2] >= 3)))]
    for row in RATE_TEST[ids, :]:
        predicted_rating = CF.pred(user_id, row[1])
        if predicted_rating >= 3 and row[1] in real_items:
            correct_items_count += 1
    user_id += 1
accuracy = correct_items_count/real_items_user_like_count
print('The accuracy of Collaborative Filtering: {0}/{1} = {2}'.format(correct_items_count, real_items_user_like_count, accuracy))

In [ ]:
from sklearn.metrics import recall_score, precision_score
x = list(set(get_rating_base_data().iloc[:,1]))
x = pd.DataFrame(columns=x)
# lấy dữ liệu của những iteam thực sự được người dùng thích
zipper = np.ones(len(real_items_1[:, 1]))
dict_ = dict(zip(real_items_1[:, 1], zipper))
y_real = x.append(dict_,ignore_index=True).fillna(0).values

# lấy dữ liệu của những item predict
zipper = np.ones(len(predicted_items))
dict_ = dict(zip(predicted_items, zipper))
y_pred = x.append(dict_,ignore_index=True).fillna(0).values

recall = recall_score(y_real, y_pred, average='macro')
precision = precision_score(y_real, y_pred, average='macro')

print('Chỉ số recall: {}\nChỉ số precision: {}'.format(recall, precision))

In [ ]:
# DF
RATE_TRAIN = get_rating_base_data().values # convert to matrix
RATE_TEST = get_rating_test_data().values # convert to matrix

RATE_TRAIN[:, :2] -= 1 # start from 0
RATE_TEST[:, :2] -= 1
USERS = get_users_data()
DF = DF(USERS, RATE_TRAIN, 25)
DF.fit()

print("Ma trận tương đồng thông tin")
print(DF.similarities)
print("Số hàng của ma trận:", DF.similarities.shape[0])
print("Số cột của ma trận: ", DF.similarities.shape[1])

ids = np.where(RATE_TEST[:, 0] == 0)[0].astype("int32")
real_items_1 = RATE_TEST[(np.where((RATE_TEST[:, 0] == 0) & (RATE_TEST[:, 2] >= 3)))]
correct_predicted_items = []

for row in RATE_TEST[ids, :]:
    predicted_rating = DF.pred(0, row[1])
    if predicted_rating >= 3 and row[1] in real_items_1:
        correct_predicted_items.append(row[1])


print("Những items user 1 thật sự thích       : ", real_items_1[:, 1])
print("Những items user 1 được dự đoán thích  : ", correct_predicted_items)

n_test = RATE_TEST.shape[0]
correct_items_count = 0
real_items_user_like_count = len(np.where(RATE_TEST[:, 2] >= 3)[0].astype(np.int32))

user_id = 0
while user_id < DF.n_users:
    ids = np.where(RATE_TEST[:, 0] == user_id)[0].astype("int32")
    real_items = RATE_TEST[(np.where((RATE_TEST[:, 0] == user_id) & (RATE_TEST[:, 2] >= 3)))]
    for row in RATE_TEST[ids, :]:
        predicted_rating = DF.pred(user_id, row[1])
        if predicted_rating >= 3 and row[1] in real_items:
            correct_items_count = correct_items_count + 1
    user_id = user_id + 1

print("Độ chính xác của Demographic Filtering :", correct_items_count / real_items_user_like_count)

In [ ]:
# # Perceptron
# ids = np.where(RATE_TEST[:, 0] == 0)[0].astype("int32")

# MATRIX_DF = []
# MATRIX_CF = []

# for row in RATE_TEST[ids, :]:
#     p_cf = CF.pred(0, row[1])
#     p_df = DF.pred(u=0, i=row[2])
#     MATRIX_CF.append([0, row[1], p_cf])
#     MATRIX_DF.append([0, row[1], p_df])
# MATRIX_CF = np.asarray(MATRIX_CF)
# MATRIX_DF = np.asarray(MATRIX_DF)

# CF_predicted = np.asanyarray(MATRIX_CF[:, 2])
# DF_predicted = MATRIX_DF[:, 2]
# true_rating = RATE_TEST[ids, 2]

# dataset = np.c_[CF_predicted, DF_predicted, true_rating]

# # print("Ma trận dự đoán đánh giá CF, DF, True Rating")
# # print(dataset)

# PLA = Perceptron(dataset, 0.003, len(ids))
# PLA.fit()
# predicted_ratings_pla = PLA.predict()

# print("Dự đoán đánh giá sau khi được điều chỉnh")
# print(np.round(predicted_ratings_pla, 3))


In [ ]:
ratings = get_rating_base_data()
df_train = pd.pivot_table(ratings, values='rating', index=['user_id'],
                    columns=['item_id'], aggfunc=np.mean).fillna(0)   

In [ ]:
# mapping:
#   - rating > 3 : 1 
#   - 0 < rating < 3: 0
#   - rating = 0 : -1
cond = lambda x: 1 if x >= 3 else (-1 if x == 0 else 0)
df_train = df_train.applymap(cond) 

In [ ]:
# Split the ratings data frame into 75% training and 25% testing
trainingset, testset = train_test_split(ratings, train_size=0.75)

# Convert training and test data into Numpy arrays
trainingset = np.array(trainingset, dtype='int')
testset = np.array(testset, dtype='int')

# Collect the total number of movies and users in order to then make a matrix of the data
config.nb_users = int(max(max(trainingset[:, 0]), max(testset[:, 0])))
config.nb_movies = int(max(max(trainingset[:, 1]), max(testset[:, 1])))

In [ ]:
# Function to convert the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []  # initialise list
    for id_users in range(1, config.nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(config.nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data


# Convert training and test sets into arrays
trainingset = convert(trainingset)
testset = convert(testset)

In [ ]:
# Convert training and test sets into torch sensors
training_set = torch.FloatTensor(trainingset)
test_set = torch.FloatTensor(testset)

In [ ]:
# Number of users
config.nb_users = DF.n_users
# Number of items
config.nb_items = DF.n_items
# Number of movies is the number of visible units
config.n_vis = len(training_set[0])
# This tunable parameter is the number of features that we want to detect (number of hidden units)
config.n_hid = 100

In [ ]:
# Create the model object RBM()
rbm = RBM(config.n_vis, config.n_hid)

config.batch_size_ = 512  # set batch size to be 512 (tunable)
reconerr = []  # keep track of reconstruction error
config.nb_epoch = 50  # run for 50 epochs

# RMSE
rmse = []

# Train the RBM
# First for loop - go through every single epoch
for epoch in range(1, config.nb_epoch + 1):
    train_recon_error = 0  # RMSE reconstruction error initialized to 0 at the beginning of training
    s = 0.  # a counter (float type)

    # Second for loop - go through every single user
    # Lower bound is 0, upper bound is (nb_users - batch_size_), batch_size_ is the step of each batch (512)
    # The 1st batch is for user with ID = 0 to user with ID = 511
    for id_user in range(0, config.nb_users - config.batch_size_, config.batch_size_):

        # At the beginning, v0 = vk. Then we update vk
        vk = training_set[id_user:id_user + config.batch_size_]
        v0 = training_set[id_user:id_user + config.batch_size_]
        ph0, _ = rbm.sample_h(v0)

        # Third for loop - perform contrastive divergence
        for k in range(10):
            _, hk = rbm.sample_h(vk)
            _, vk = rbm.sample_v(hk)

            # We don't want to learn when there is no rating by the user, and there is no update when rating = -1
            vk[v0 < 0] = v0[v0 < 0]

        phk, _ = rbm.sample_h(vk)

        # Calculate the loss using contrastive divergence
        rbm.train(v0, vk, ph0, phk)

        # Compare vk updated after the training to v0 (the target)
        train_recon_error += torch.sqrt(torch.mean((v0[v0 >= 0] - vk[v0 >= 0])**2))
        s += 1.

    # Update RMSE reconstruction error
    reconerr.append(train_recon_error / s)

    print('Epoch: ' + str(epoch) + '- RMSE Reconstruction Error: ' + str(train_recon_error.data.numpy() / s))
    wandb.log({"Train RMSE": train_recon_error.data.numpy() / s})

In [24]:
torch.rand(1, config.n_hid)

tensor([[0.7093, 0.8605, 0.2075, 0.1905, 0.4305, 0.3963, 0.5160, 0.0270, 0.7015,
         0.7043, 0.1175, 0.2774, 0.5912, 0.8797, 0.1555, 0.3118, 0.1021, 0.1235,
         0.3839, 0.0044, 0.1565, 0.4884, 0.5836, 0.6928, 0.7131, 0.1555, 0.4190,
         0.0509, 0.1111, 0.1319, 0.8891, 0.8353, 0.0730, 0.0373, 0.3155, 0.9898,
         0.3819, 0.8308, 0.1921, 0.7566, 0.8650, 0.5129, 0.3271, 0.6680, 0.4904,
         0.0596, 0.5981, 0.4602, 0.4618, 0.6667, 0.9513, 0.2119, 0.7331, 0.9846,
         0.1797, 0.8465, 0.1183, 0.1834, 0.8429, 0.5681, 0.8861, 0.6708, 0.9481,
         0.1706, 0.5040, 0.0463, 0.0209, 0.4391, 0.9406, 0.7443, 0.3492, 0.1972,
         0.1951, 0.3796, 0.2159, 0.9157, 0.7877, 0.3728, 0.8788, 0.2852, 0.9288,
         0.3547, 0.9041, 0.1623, 0.4421, 0.5251, 0.3797, 0.8676, 0.0015, 0.7613,
         0.4114, 0.6531, 0.7920, 0.4697, 0.9334, 0.1087, 0.0456, 0.7447, 0.0200,
         0.1766]])